In [2]:
import os
import pandas as pd
import gzip
import numpy as np
from itertools import product
from joblib import Parallel, delayed
from tqdm.auto import tqdm
from collections import defaultdict, Counter, OrderedDict

/opt/conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Function to read FASTA file and yield name and sequence
def fasta_iter(fasta_file):
    with gzip.open(fasta_file, 'rt') as file:
        name = None
        seqs = []
        for line in file:
            line = line.strip()
            if line.startswith(">"):
                if name:
                    yield name, ''.join(seqs)
                name = line[1:]  # Skip '>'
                seqs = []
            else:
                seqs.append(line)
        if name:
            yield name, ''.join(seqs)  # Yield the last entry

# Function to read FASTQ file and yield name and sequence
def fastq_iter(fastq_file):
    with gzip.open(fastq_file, 'rt') as file:
        while True:
            name = file.readline().strip()[1:]  # Skip '@' in FASTQ
            seq = file.readline().strip()
            file.readline()  # Skip the '+' line
            file.readline()  # Skip the quality line
            if not name or not seq:
                break
            yield name, seq

# Function to generate feature mapping for k-mers with sequences as keys
def generate_feature_mapping(kmer_len):
    BASE_COMPLEMENT = {"A": "T", "T": "A", "G": "C", "C": "G"}
    kmer_hash = {}
    kmer_names = []
    counter = 0
    for kmer in product("ATGC", repeat=kmer_len):
        kmer = ''.join(kmer)
        rev_compl = ''.join([BASE_COMPLEMENT[x] for x in reversed(kmer)])  # Generate reverse complement
        if kmer not in kmer_hash and rev_compl not in kmer_hash:
            # Use canonical k-mer (lexicographically smaller of kmer and rev_compl)
            canonical_kmer = min(kmer, rev_compl)
            kmer_hash[canonical_kmer] = counter
            kmer_names.append(canonical_kmer)  # Store the k-mer for naming features
            counter += 1
    return kmer_hash, kmer_names, counter
    
# Function to detect file type based on file extension
def detect_file_type(file_path):
    ext = os.path.splitext(file_path)[-1].lower()  # Get file extension and convert to lowercase
    # Handle gzipped files
    if ext == '.gz':
        ext = os.path.splitext(file_path[:-3])[-1].lower()  # Check the extension before ".gz"
        
    if ext in ['.fasta', '.fa', '.fna']:
        return 'fasta'
    elif ext in ['.fastq', '.fq']:
        return 'fastq'
    else:
        raise ValueError("Unsupported file extension. Please provide a valid FASTA or FASTQ file.")
        
# Function to generate k-mer features from FASTA or FASTQ
def generate_kmer_features(file_path, length_threshold, kmer_len, split=False, split_threshold=0):
    
    # Automatically detect file type from file extension
    file_type = detect_file_type(file_path)
    
    # Select the appropriate file iterator based on the file type
    if file_type == 'fasta':
        seq_iter = fasta_iter(file_path)
    elif file_type == 'fastq':
        seq_iter = fastq_iter(file_path)
    else:
        raise ValueError("Unsupported file type. Use 'fasta' or 'fastq'.")

    def seq_list():
        for h, seq in seq_iter:
            if not split:
                yield h, seq
            elif len(seq) >= split_threshold:
                half = len(seq) // 2
                yield (h + '_1', seq[:half])
                yield (h + '_2', seq[half:])

    kmer_dict, kmer_names, nr_features = generate_feature_mapping(kmer_len)
    composition = OrderedDict()

    for h, seq in seq_list():
        if len(seq) < length_threshold:
            continue
        norm_seq = str(seq).upper()
        kmers = [kmer_dict[norm_seq[i:i+kmer_len]]
                 for i in range(len(norm_seq) - kmer_len + 1)
                 if norm_seq[i:i+kmer_len] in kmer_dict]  # Ignore kmers with non-canonical bases
        composition[h] = np.bincount(np.array(kmers, dtype=np.int64), minlength=nr_features)

    # Create DataFrame with k-mer names as column headers
    df = pd.DataFrame.from_dict(composition, orient='index', dtype=float, columns=kmer_names)

    df = df.apply(lambda x: x + 1e-5)  # Smoothing
    df = df.div(df.sum(axis=1), axis=0)  # Normalize by row (sequence)

    return df


# Contigs

## Feature

In [4]:
fasta_file = './contigs/anonymous_gsa.fasta.gz'  # Updated file path to your uploaded file
kmer_len = 5  # Original k-mer length
length_threshold = 100  # Example sequence length threshold

# Generate spaced k-mer features
kmer_df = generate_kmer_features(fasta_file, length_threshold, kmer_len)
print(kmer_df)

                 AAAAA         AAAAT         AAAAG         AAAAC  \
S0C0      7.692005e-08  7.692005e-08  7.692005e-08  7.692005e-08   
S0C1      1.149358e-07  1.149358e-07  1.149369e-02  1.149358e-07   
S0C2      9.345347e-08  9.345347e-08  9.345347e-08  9.345441e-03   
S0C3      6.289106e-08  6.289106e-08  6.289106e-08  6.289106e-08   
S0C4      4.291794e-03  4.291751e-08  4.291751e-08  1.287530e-02   
...                ...           ...           ...           ...   
S0C47884  5.714119e-08  5.714176e-03  5.714119e-08  5.714119e-08   
S0C47885  9.090577e-03  9.090577e-03  9.090486e-08  1.818106e-02   
S0C47886  1.449168e-07  1.449168e-07  1.449182e-02  1.449168e-07   
S0C47887  2.040603e-07  2.040603e-07  2.040603e-07  2.040603e-07   
S0C47888  7.873698e-08  7.873698e-08  7.873698e-08  7.873698e-08   

                 AAATA         AAATT         AAATG         AAATC  \
S0C0      7.692005e-08  7.692005e-08  7.692005e-08  7.692005e-08   
S0C1      1.149358e-07  1.149358e-07  1.149358e

## Label

In [5]:
label_path = "./contigs/gsa_mapping.tsv.gz"
label_df = pd.read_csv(label_path, sep='\t')
label_df = label_df[["genome_id"]]
label_df

,genome_id
0,Genome4.1
1,Genome8.0
2,Genome10.0
3,Genome6.0
4,Genome23.0
...,...
47884,Genome18.0
47885,Genome10.1
47886,Genome8.0
47887,Genome3.0


In [6]:
# Tampilkan statistik deskriptif dari data fitur
print("Statistik Deskriptif Data Fitur:")
print(kmer_df.describe())

# Tampilkan distribusi kelas dari data label
print("\nDistribusi Kelas pada Data Label:")
print(label_df.value_counts())

Statistik Deskriptif Data Fitur:
              AAAAA         AAAAT         AAAAG         AAAAC         AAATA  \
count  4.788900e+04  4.788900e+04  4.788900e+04  4.788900e+04  4.788900e+04   
mean   4.466781e-03  3.332193e-03  3.237534e-03  2.674919e-03  2.457743e-03   
std    9.225107e-03  6.215612e-03  5.737540e-03  4.936308e-03  5.346345e-03   
min    6.067942e-09  7.535766e-09  6.313111e-09  6.341134e-09  7.153050e-09   
25%    7.246108e-08  7.246108e-08  7.352664e-08  7.246108e-08  6.992757e-08   
50%    9.999488e-08  1.010049e-07  1.052575e-07  9.999488e-08  9.090486e-08   
75%    6.243485e-03  5.847837e-03  5.987900e-03  4.804796e-03  1.305485e-03   
max    1.111049e-01  6.848848e-02  5.376059e-02  4.999590e-02  6.666114e-02   

              AAATT         AAATG         AAATC         AAAGA         AAAGT  \
count  4.788900e+04  4.788900e+04  4.788900e+04  4.788900e+04  4.788900e+04   
mean   2.447652e-03  1.975500e-03  2.383826e-03  2.809300e-03  1.821053e-03   
std    5.205263e-0

## Classifier

In [7]:
from sklearn.model_selection import train_test_split
import time
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report


### Pembagian Data

In [8]:
features = kmer_df
labels = label_df

# Pisahkan data menjadi training dan testing set (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

print("Jumlah data latih per kelas:")
print(np.unique(y_train, return_counts=True))

print("\nJumlah data uji per kelas:")
print(np.unique(y_test, return_counts=True))

Jumlah data latih per kelas:
(array(['Genome10.0', 'Genome10.1', 'Genome11.0', 'Genome12.0',
       'Genome13.0', 'Genome14.0', 'Genome15.0', 'Genome16.0',
       'Genome17.0', 'Genome18.0', 'Genome19.0', 'Genome2.0',
       'Genome20.0', 'Genome22.0', 'Genome23.0', 'Genome24.0',
       'Genome3.0', 'Genome4.0', 'Genome4.1', 'Genome5.0', 'Genome6.0',
       'Genome7.0', 'Genome8.0', 'Genome9.0'], dtype=object), array([1408, 3498,   23,  257, 1061, 2160, 2456,  193,  971, 1457,    1,
       3613,  475, 4622, 1265,  250, 3906, 2526,  822,  198, 2845,  331,
       3775,  198]))

Jumlah data uji per kelas:
(array(['Genome10.0', 'Genome10.1', 'Genome11.0', 'Genome12.0',
       'Genome13.0', 'Genome14.0', 'Genome15.0', 'Genome16.0',
       'Genome17.0', 'Genome18.0', 'Genome2.0', 'Genome20.0',
       'Genome22.0', 'Genome23.0', 'Genome24.0', 'Genome3.0', 'Genome4.0',
       'Genome4.1', 'Genome5.0', 'Genome6.0', 'Genome7.0', 'Genome8.0',
       'Genome9.0'], dtype=object), array([ 380,  846,

### Random Forest

In [9]:
# List model yang akan digunakan
models = {
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "Logistic Regression": LogisticRegression(random_state=42),
    "Support Vector Machine": SVC(random_state=42),
    "K-Nearest Neighbors": KNeighborsClassifier(),
    #"Gradient Boosting": GradientBoostingClassifier(n_estimators=100, random_state=42),
    #"MLP Classifier": MLPClassifier(random_state=42, max_iter=500)
}

# Menyimpan hasil metrik dan waktu komputasi
results = []

# Loop melalui semua model dan evaluasi performanya
for model_name, model in models.items():
    start_time = time.time()  # Mulai penghitungan waktu
    
    # Latih model
    model.fit(X_train, y_train)
    
    # Prediksi menggunakan data testing
    y_pred = model.predict(X_test)
    
    # Hitung waktu komputasi
    elapsed_time = time.time() - start_time
    
    # Evaluasi model dengan berbagai metrik
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    
    # Simpan hasil dalam dictionary
    results.append({
        "Model": model_name,
        "Accuracy": accuracy,
        "Precision": precision,
        "Recall": recall,
        "F1 Score": f1,
        "Time (seconds)": elapsed_time
    })
    
    # Tampilkan hasil classification report untuk setiap model
    print(f"\nClassification Report untuk {model_name}:")
    print(classification_report(y_test, y_pred))

# Tampilkan hasil dalam bentuk dataframe untuk mempermudah pembacaan
import pandas as pd
results_df = pd.DataFrame(results)
results_df

/opt/conda/lib/python3.11/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Classification Report untuk Random Forest:
              precision    recall  f1-score   support

  Genome10.0       0.08      0.01      0.01       380
  Genome10.1       0.39      0.77      0.52       846
  Genome11.0       0.00      0.00      0.00         3
  Genome12.0       0.00      0.00      0.00        59
  Genome13.0       0.75      0.01      0.02       277
  Genome14.0       0.63      0.63      0.63       538
  Genome15.0       0.71      0.36      0.48       631
  Genome16.0       0.00      0.00      0.00        54
  Genome17.0       0.89      0.03      0.06       258
  Genome18.0       0.93      0.75      0.83       356
   Genome2.0       0.56      0.53      0.54       883
  Genome20.0       0.00      0.00      0.00       110
  Genome22.0       0.48      0.78      0.59      1142
  Genome23.0       0.79      0.06      0.11       326
  Genome24.0       0.00      0.00      0.00        59
   Genome3.0       0.48      0.71      0.58      1005
   Genome4.0       0.62      0.78    

/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.11/site-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning


Classification Report untuk Logistic Regression:
              precision    recall  f1-score   support

  Genome10.0       0.00      0.00      0.00       380
  Genome10.1       0.33      0.72      0.46       846
  Genome11.0       0.00      0.00      0.00         3
  Genome12.0       0.00      0.00      0.00        59
  Genome13.0       0.00      0.00      0.00       277
  Genome14.0       0.90      0.14      0.25       538
  Genome15.0       0.89      0.23      0.37       631
  Genome16.0       0.00      0.00      0.00        54
  Genome17.0       0.00      0.00      0.00       258
  Genome18.0       0.00      0.00      0.00       356
   Genome2.0       0.58      0.54      0.56       883
  Genome20.0       0.00      0.00      0.00       110
  Genome22.0       0.41      0.87      0.55      1142
  Genome23.0       0.00      0.00      0.00       326
  Genome24.0       0.00      0.00      0.00        59
   Genome3.0       0.48      0.81      0.60      1005
   Genome4.0       0.67      0.

/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.11/site-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning


Classification Report untuk Support Vector Machine:
              precision    recall  f1-score   support

  Genome10.0       0.00      0.00      0.00       380
  Genome10.1       0.51      0.87      0.64       846
  Genome11.0       0.00      0.00      0.00         3
  Genome12.0       0.00      0.00      0.00        59
  Genome13.0       0.65      0.41      0.50       277
  Genome14.0       0.70      0.83      0.76       538
  Genome15.0       0.80      0.74      0.77       631
  Genome16.0       0.00      0.00      0.00        54
  Genome17.0       0.85      0.60      0.70       258
  Genome18.0       0.96      0.85      0.90       356
   Genome2.0       0.70      0.75      0.72       883
  Genome20.0       0.67      0.31      0.42       110
  Genome22.0       0.72      0.82      0.77      1142
  Genome23.0       0.70      0.62      0.66       326
  Genome24.0       0.00      0.00      0.00        59
   Genome3.0       0.70      0.80      0.75      1005
   Genome4.0       0.70     

/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.11/site-packages/sklearn/neighbors/_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning:


Classification Report untuk K-Nearest Neighbors:
              precision    recall  f1-score   support

  Genome10.0       0.31      0.01      0.02       380
  Genome10.1       0.52      0.15      0.23       846
  Genome11.0       0.00      0.00      0.00         3
  Genome12.0       0.00      0.00      0.00        59
  Genome13.0       0.00      0.00      0.00       277
  Genome14.0       0.29      0.07      0.11       538
  Genome15.0       0.68      0.23      0.34       631
  Genome16.0       0.00      0.00      0.00        54
  Genome17.0       0.57      0.05      0.09       258
  Genome18.0       0.05      1.00      0.10       356
  Genome19.0       0.00      0.00      0.00         0
   Genome2.0       0.32      0.06      0.10       883
  Genome20.0       0.00      0.00      0.00       110
  Genome22.0       0.77      0.02      0.04      1142
  Genome23.0       0.75      0.01      0.02       326
  Genome24.0       0.00      0.00      0.00        59
   Genome3.0       0.56      0.

/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


,Model,Accuracy,Precision,Recall,F1 Score,Time (seconds)
0,Random Forest,0.544790,0.537341,0.544790,0.490294,64.311304
1,Logistic Regression,0.501148,0.448129,0.501148,0.423033,60.596184
2,Support Vector Machine,0.710378,0.672095,0.710378,0.677247,391.932779
3,K-Nearest Neighbors,0.208185,0.496038,0.208185,0.220186,1.435849


In [10]:
# List model yang akan digunakan
models = {
    # "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    # "Logistic Regression": LogisticRegression(random_state=42),
    # "Support Vector Machine": SVC(random_state=42),
    # "K-Nearest Neighbors": KNeighborsClassifier(),
    # "Gradient Boosting": GradientBoostingClassifier(n_estimators=100, random_state=42),
    "MLP Classifier": MLPClassifier(random_state=42, max_iter=100)
}

# Menyimpan hasil metrik dan waktu komputasi
results = []

# Loop melalui semua model dan evaluasi performanya
for model_name, model in models.items():
    start_time = time.time()  # Mulai penghitungan waktu
    
    # Latih model
    model.fit(X_train, y_train)
    
    # Prediksi menggunakan data testing
    y_pred = model.predict(X_test)
    
    # Hitung waktu komputasi
    elapsed_time = time.time() - start_time
    
    # Evaluasi model dengan berbagai metrik
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    
    # Simpan hasil dalam dictionary
    results.append({
        "Model": model_name,
        "Accuracy": accuracy,
        "Precision": precision,
        "Recall": recall,
        "F1 Score": f1,
        "Time (seconds)": elapsed_time
    })
    
    # Tampilkan hasil classification report untuk setiap model
    print(f"\nClassification Report untuk {model_name}:")
    print(classification_report(y_test, y_pred))

dff = pd.DataFrame(results)
dff

/opt/conda/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1105: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no p


Classification Report untuk MLP Classifier:
              precision    recall  f1-score   support

  Genome10.0       0.00      0.00      0.00       380
  Genome10.1       0.52      0.81      0.64       846
  Genome11.0       0.00      0.00      0.00         3
  Genome12.0       0.00      0.00      0.00        59
  Genome13.0       0.56      0.45      0.50       277
  Genome14.0       0.68      0.82      0.74       538
  Genome15.0       0.77      0.74      0.75       631
  Genome16.0       0.33      0.02      0.04        54
  Genome17.0       0.78      0.71      0.74       258
  Genome18.0       0.80      0.88      0.84       356
   Genome2.0       0.73      0.72      0.72       883
  Genome20.0       0.45      0.37      0.41       110
  Genome22.0       0.75      0.78      0.77      1142
  Genome23.0       0.65      0.63      0.64       326
  Genome24.0       0.00      0.00      0.00        59
   Genome3.0       0.70      0.77      0.73      1005
   Genome4.0       0.70      0.88   

/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


,Model,Accuracy,Precision,Recall,F1 Score,Time (seconds)
0,MLP Classifier,0.704531,0.658619,0.704531,0.67502,357.944669


In [11]:
hasil = pd.concat([results_df, dff], ignore_index=True)
hasil

,Model,Accuracy,Precision,Recall,F1 Score,Time (seconds)
0,Random Forest,0.544790,0.537341,0.544790,0.490294,64.311304
1,Logistic Regression,0.501148,0.448129,0.501148,0.423033,60.596184
2,Support Vector Machine,0.710378,0.672095,0.710378,0.677247,391.932779
3,K-Nearest Neighbors,0.208185,0.496038,0.208185,0.220186,1.435849
4,MLP Classifier,0.704531,0.658619,0.704531,0.675020,357.944669


In [12]:
hasil.to_csv('../../result/Kmers_camisim.csv', index=False)

# Reads

## Feature

In [9]:
read_file = './reads/anonymous_reads.fq.gz'  # Updated file path to your uploaded file
kmer_len = 4  # Original k-mer length
length_threshold = 100  # Example sequence length threshold

# Generate spaced k-mer features
featureR_df = generate_kmer_features(read_file, length_threshold, kmer_len)
print(featureR_df)

                     AAAA          AAAT          AAAG          AAAC  \
S0R0/1       1.162772e-07  1.162772e-07  1.162772e-07  1.162784e-02   
S0R0/2       1.449247e-07  1.449247e-07  1.449247e-07  1.449247e-07   
S0R1/1       1.351327e-07  1.351327e-07  1.351327e-07  1.351340e-02   
S0R1/2       1.190457e-07  1.190457e-07  1.190457e-07  1.190457e-07   
S0R2/1       1.851805e-07  1.851805e-07  1.851805e-07  1.851805e-07   
...                   ...           ...           ...           ...   
S0R333324/2  1.666629e-07  1.666629e-07  1.666629e-07  1.666629e-07   
S0R333325/1  1.136346e-07  1.136346e-07  1.136346e-07  1.136346e-07   
S0R333325/2  1.333309e-07  1.333309e-07  1.333322e-02  1.333322e-02   
S0R333326/1  1.408424e-07  1.408424e-07  1.408438e-02  1.408424e-07   
S0R333326/2  3.260832e-02  2.173892e-02  1.086951e-02  1.086951e-02   

                     AATA          AATT          AATG          AATC  \
S0R0/1       1.162772e-07  1.162772e-07  1.162772e-07  1.162772e-07   
S0R0/

In [13]:
labelR_path = "./reads/reads_mapping.tsv.gz"
labelR_df = pd.read_csv(labelR_path, sep='\t')
labelR_df = labelR_df[["genome_id"]]
labelR_df

,genome_id
0,Genome19.0
1,Genome19.0
2,Genome19.0
3,Genome19.0
4,Genome18.0
...,...
666649,Genome18.0
666650,Genome19.0
666651,Genome19.0
666652,Genome19.0


In [16]:
features = featureR_df
labels = labelR_df

# Pisahkan data menjadi training dan testing set (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

print("Jumlah data latih per kelas:")
print(np.unique(y_train, return_counts=True))

print("\nJumlah data uji per kelas:")
print(np.unique(y_test, return_counts=True))

Jumlah data latih per kelas:
(array(['Genome10.0', 'Genome10.1', 'Genome11.0', 'Genome12.0',
       'Genome13.0', 'Genome14.0', 'Genome15.0', 'Genome16.0',
       'Genome17.0', 'Genome18.0', 'Genome19.0', 'Genome2.0',
       'Genome20.0', 'Genome22.0', 'Genome23.0', 'Genome24.0',
       'Genome3.0', 'Genome4.0', 'Genome4.1', 'Genome5.0', 'Genome6.0',
       'Genome7.0', 'Genome8.0', 'Genome9.0'], dtype=object), array([  2843,   9214,     37,    438,   2081,   8187,   5417,    338,
         1936,  69103, 367831,  12720,    853,  13194,   2856,    444,
         9619,   5141,   1501,    340,   7383,    656,  10800,    391]))

Jumlah data uji per kelas:
(array(['Genome10.0', 'Genome10.1', 'Genome11.0', 'Genome12.0',
       'Genome13.0', 'Genome14.0', 'Genome15.0', 'Genome16.0',
       'Genome17.0', 'Genome18.0', 'Genome19.0', 'Genome2.0',
       'Genome20.0', 'Genome22.0', 'Genome23.0', 'Genome24.0',
       'Genome3.0', 'Genome4.0', 'Genome4.1', 'Genome5.0', 'Genome6.0',
       'Genome7.0'

In [17]:
# Inisialisasi model Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Latih model
rf_model.fit(X_train, y_train)

# Prediksi menggunakan data testing
y_pred = rf_model.predict(X_test)

# Tampilkan hasil akurasi dan classification report
print("\nAkurasi Model:")
print(accuracy_score(y_test, y_pred))

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

/opt/conda/lib/python3.11/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)



Akurasi Model:
0.6923746165557897

Classification Report:


/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

  Genome10.0       0.00      0.00      0.00       759
  Genome10.1       0.00      0.00      0.00      2264
  Genome11.0       0.00      0.00      0.00         9
  Genome12.0       0.00      0.00      0.00       106
  Genome13.0       0.00      0.00      0.00       491
  Genome14.0       0.00      0.00      0.00      2001
  Genome15.0       0.00      0.00      0.00      1323
  Genome16.0       0.00      0.00      0.00        94
  Genome17.0       0.00      0.00      0.00       446
  Genome18.0       0.14      0.00      0.00     17141
  Genome19.0       0.69      1.00      0.82     92345
   Genome2.0       0.00      0.00      0.00      3170
  Genome20.0       0.00      0.00      0.00       189
  Genome22.0       0.00      0.00      0.00      3268
  Genome23.0       0.00      0.00      0.00       730
  Genome24.0       0.00      0.00      0.00       118
   Genome3.0       0.00      0.00      0.00      2399
   Genome4.0       0.00    

/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
